In [7]:
import csv
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score

file_dir_train = os.path.join(os.getcwd(), 'train.csv')
file_dir_test = os.path.join(os.getcwd(), 'test.csv')
df_train = pd.read_csv(file_dir_train)
df_test = pd.read_csv(file_dir_test)

X_train = np.array(df_train.iloc[:,1:])
y_train = np.array(df_train.iloc[:,0])
X_test= np.array(df_test.iloc[:,1:])
y_test = np.array(df_test.iloc[:,0])

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

mlp = MLPClassifier(max_iter=1)

parameter_space = {
    'hidden_layer_sizes' : [(10,), (20,)],
    'activation' : ['relu', 'tanh'],
    'solver' : ['sgd', 'adam'],
    'alpha' : [0.0001, 0.05],
    'learning_rate' : [ 'constant' , 'adaptive']
}

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)

In [ ]:
print('Best parameters found:\n', clf.best_params_)

In [6]:
predictions = clf.predict(X_test)
accuracy_score(y_test, predictions)

0.6989333333333333